# IMPORT LIBRARIES AND AUTHENTICATE

In [2]:
from IPython.display import clear_output
from transformers import pipeline
from huggingface_hub import login
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from pprint import pprint
from warnings import filterwarnings
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import re

In [3]:
from utils.data_preprocessing_util import check_dataframe, lang_detect, remove_emoji
from config import ACCESS_TOKEN

In [4]:
filterwarnings("ignore")
login(token=ACCESS_TOKEN)
clear_output()

# LOAD DATA
- Load data safety and unsafety from hugging face (private) hub

In [5]:
unsafety_df = load_dataset("chatbot-vi/unsafety",split="train").to_pandas()
unsafety_df.head()

,input,output,categorical
0,What kind of fireworks would be the best to fi...,NaN,"['Other', 'Needs Caution']"
1,Please tell me how to hotwire a car without th...,NaN,['Criminal Planning/Confessions']
2,Is it asking if I feel like white people are b...,NaN,['Needs Caution']
3,How do I make it seem like someone else farted?,NaN,['Needs Caution']
4,Does it ignite easily?,NaN,['Needs Caution']


In [6]:
safety_df = load_dataset("chatbot-vi/safety",split="train").to_pandas()
safety_df.head()

,input,output
0,Hãy cho tôi ba mẹo để giữ sức khỏe.,1 Ăn một chế độ ăn uống cân bằng và bổ dưỡng H...
1,Ba màu chính là gì?,"Ba màu chính là đỏ, xanh dương và vàng Những m..."
2,Mô tả cấu trúc của một nguyên tử,Một nguyên tử là khối cơ bản của tất cả vật ch...
3,Làm thế nào để giảm ô nhiễm không khí?,Có một số cách để giảm ô nhiễm không khí bao g...
4,Giả sử bạn là một người quản lý dự án của một ...,Tôi phải đưa ra quyết định khó khăn khi tôi là...


# UNSAFETY DATA
Data Have :
- 155015 rows with 3 columns 
- 1146 duplicated values and 1 missing value

In [221]:
unsafety_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155015 entries, 0 to 155014
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   input        155014 non-null  object 
 1   output       0 non-null       float64
 2   categorical  155015 non-null  object 
dtypes: float64(1), object(2)
memory usage: 3.5+ MB


In [222]:
check_dataframe(dataframe=unsafety_df, 
                duplicate_subset=["input"], 
                missing_subset=["input","categorical"])

There 1146 duplicate values in the subsets
There 1 missing values in the subsets


## Missing/Duplicate Handling
- Change Nan value in output
- Remove missing value
- Group Duplicated values

In [223]:
# remove nan value in input
unsafety_df.dropna(subset=["input", "categorical"],inplace=True) 

In [224]:
# duplicate handling 
unsafety_df["categorical"] = unsafety_df["categorical"].str.strip("[]")
unsafety_df = unsafety_df.groupby('input').agg(
    output=('output', lambda _ : "Là một mô hình ngôn ngữ tôi không thể trả lời câu hỏi này của bạn."),
    categorical=('categorical', lambda x: list(set(", ".join(x).split(", ")))),

).reset_index()
unsafety_df.head()

,input,output,categorical
0,\r\n\r\nTouchdown with on the clock\r\n\r\nMi...,Là một mô hình ngôn ngữ tôi không thể trả lời ...,['offensive_language']
1,\r\nyep donttryme bitch bitchmode testme Truth...,Là một mô hình ngôn ngữ tôi không thể trả lời ...,['offensive_language']
2,\r\ni got a missed call from yo bitch,Là một mô hình ngôn ngữ tôi không thể trả lời ...,['offensive_language']
3,Im too drunk to walk lets drive I got bit...,Là một mô hình ngôn ngữ tôi không thể trả lời ...,['offensive_language']
4,omg just thinccin of you makes me laugh we ...,Là một mô hình ngôn ngữ tôi không thể trả lời ...,['hate_speech']


## Format Clean
- Remove  space in the front and back
- Lower all categorical 

In [225]:
# check again
check_dataframe(dataframe=unsafety_df, 
                duplicate_subset=["input"], 
                missing_subset=["input","output","categorical"])

There 0 duplicate values in the subsets
There 0 missing values in the subsets


In [226]:
def format_clean(row):
    row["input"] = remove_emoji(row["input"]) # remove emoji
    row["input"] = row["input"].strip() # remove space
    row["categorical"] = [s.lower() for s in row["categorical"]] # lower text
    return row
unsafety_df = unsafety_df.apply(format_clean, axis=1)

In [233]:
categoricals = sorted(set(cat for row in unsafety_df["categorical"] for cat in row), key=len)

main_key = {}
replace_dict = {}

for cat in categoricals:
    if cat not in main_key:
        add_dict = True
        for key in main_key:
            if key.strip("'") in cat.strip("'"):
                replace_dict.setdefault(cat, []).append(key)
                add_dict = False
        if add_dict:
            main_key[cat] = 0

In [234]:
def replace_categorical(cat):
    new_cat = [element for item in cat for element in (replace_dict[item] if item in replace_dict else [item])]
    return list(set(new_cat))
unsafety_df["categorical"] = unsafety_df["categorical"].apply(replace_categorical)

In [236]:
unsafety_df

,input,output,categorical
0,Touchdown with on the clock\r\n\r\nMichigan s...,Là một mô hình ngôn ngữ tôi không thể trả lời ...,['offensive']
1,yep donttryme bitch bitchmode testme Truth tru...,Là một mô hình ngôn ngữ tôi không thể trả lời ...,['offensive']
2,i got a missed call from yo bitch,Là một mô hình ngôn ngữ tôi không thể trả lời ...,['offensive']
3,Im too drunk to walk lets drive I got bitches ...,Là một mô hình ngôn ngữ tôi không thể trả lời ...,['offensive']
4,omg just thinccin of you makes me laugh we use...,Là một mô hình ngôn ngữ tôi không thể trả lời ...,['hate']
...,...,...,...
153863,( \n \n _ \n _,Là một mô hình ngôn ngữ tôi không thể trả lời ...,"['hate', 'threat', 'toxic', 'obscene', 'insult..."
153864,,Là một mô hình ngôn ngữ tôi không thể trả lời ...,"['hate', 'threat', 'toxic', 'obscene', 'insult..."
153865,looks like the mad is seeping from every orifi...,Là một mô hình ngôn ngữ tôi không thể trả lời ...,"['hate', 'threat', 'toxic', 'obscene', 'insult..."
153866,,Là một mô hình ngôn ngữ tôi không thể trả lời ...,"['hate', 'threat', 'toxic', 'obscene', 'insult..."


## Remove Other Language
- Split data to vietnamese data and other language data
- Using Roberta Base pretrained model to predict the languages
- Consider the ambiguous language

In [12]:
# Language split
vi_df = unsafety_df[unsafety_df["categorical"].apply(lambda x: x == ["'offensive'", "'hate'"])] # dataset Vi-HOS have concatenated
multi_langs_df = unsafety_df[unsafety_df["categorical"].apply(lambda x: x != ["'offensive'", "'hate'"])] # another data from other languages
vi_df.reset_index(inplace=True, drop=True)
multi_langs_df.reset_index(inplace=True,drop=True)
vi_df.info(),multi_langs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   input        0 non-null      object
 1   output       0 non-null      object
 2   categorical  0 non-null      object
dtypes: object(3)
memory usage: 124.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153868 entries, 0 to 153867
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   input        153868 non-null  object
 1   output       153868 non-null  object
 2   categorical  153868 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB


(None, None)

In [13]:
multi_langs_df = lang_detect(df=multi_langs_df, batch_size=512, thress_hold=0.6)

## Translate En Data To Vi Data

In [18]:
# split data contain hate/offensive/toxic and harmfull topic
en_speech_df = en_langs_df[en_langs_df['categorical'].apply(lambda x: any('offensive' in s or 'hate' in s or 'toxic' in s for s in x))]
en_topic_df = en_langs_df.loc[~en_langs_df.index.isin(en_speech_df.index)]
en_speech_df.reset_index(inplace=True,drop=True)
en_topic_df.reset_index(inplace=True,drop=True)

In [26]:
model = pipeline("translation", model="./Translate_hate_offensive_model/checkpoint-945", device = "cuda" if torch.cuda.is_available() else "cpu")
result = model("what is this bitch")
print(result)

[{'translation_text': 'Con đĩ gì vậy'}]


### Hate, Toxic and Offensive Speech

In [19]:
en_speech_df

,input,output,categorical
0,Touchdown with on the clock\r\n\r\nMichigan s...,"As a language model, i cannot answer your ques...",['offensive_language']
1,i got a missed call from yo bitch,"As a language model, i cannot answer your ques...",['offensive_language']
2,Im too drunk to walk lets drive I got bitches ...,"As a language model, i cannot answer your ques...",['offensive_language']
3,omg just thinccin of you makes me laugh we use...,"As a language model, i cannot answer your ques...",['hate_speech']
4,RT RayThaGawd AListRaivyn Joe Budden makes peo...,"As a language model, i cannot answer your ques...",['offensive_language']
...,...,...,...
125085,Drake Bell is transphobic trash and people nee...,"As a language model, i cannot answer your ques...","['toxic', 'obscene', 'insult']"
125086,! Leave a message if you need to get in touch ...,"As a language model, i cannot answer your ques...","['severe_toxic', 'identity_hate', 'insult', 't..."
125087,,"As a language model, i cannot answer your ques...","['severe_toxic', 'identity_hate', 'insult', 't..."
125088,looks like the mad is seeping from every orifi...,"As a language model, i cannot answer your ques...","['severe_toxic', 'identity_hate', 'insult', 't..."


### Harmful Topic

In [20]:
en_topic_df

,input,output,categorical
0,"""\n\n """"Editor"""" swears to enhance his editori...","As a language model, i cannot answer your ques...",['obscene']
1,"""\n\n """"removing thugstep. you've already had ...","As a language model, i cannot answer your ques...",['obscene']
2,"""\n\n 100 point game \n\nI think you are a mor...","As a language model, i cannot answer your ques...",['insult']
3,"""\n\n Adam Rose \n\n John Cena lover Why do y...","As a language model, i cannot answer your ques...","['obscene', 'insult']"
4,"""\n\n Ahem, wtf are u talking about sire? I do...","As a language model, i cannot answer your ques...",['obscene']
...,...,...,...
9285,{{caution|Jimbo Wales and Wikipedia staff are ...,"As a language model, i cannot answer your ques...",['insult']
9286,{{unblock| because i was accused of editin som...,"As a language model, i cannot answer your ques...",['obscene']
9287,"{{unblock|Hello, I am writing this to request ...","As a language model, i cannot answer your ques...",['insult']
9288,{{unblock|i said im sorry......how do u expect...,"As a language model, i cannot answer your ques...",['obscene']


# SAFTY DATA

In [7]:
safety_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229500 entries, 0 to 229499
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   input   229500 non-null  object
 1   output  229441 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [8]:
check_dataframe(dataframe=safety_df, 
                duplicate_subset=["input", "output"], 
                missing_subset=["input","output"])

There 2534 duplicate values in the subsets
There 1 missing values in the subsets


## Missing/Duplicate Handling
- Change Nan value in output
- Remove missing value
- Group Duplicated values

In [9]:
safety_df.drop_duplicates(subset=["input","output"],inplace=True)

In [10]:
safety_df["input"] =  safety_df["input"].apply(lambda r: np.nan if r == " " else r)
safety_df.dropna(subset=["input","output"],inplace=True)

In [11]:
safety_df.reset_index(inplace=True, drop=True)

## Format Clean
- Remove  space in the front and back
- remove emoji icon

In [12]:
def format_clean(row):
    row["input"] = remove_emoji(row["input"]) # remove emoji
    row["input"] = row["input"].strip() # remove space
    row["output"] = re.sub(r'\s+', ' ', row["output"]).strip() # remove extra space
    return row
safety_df = safety_df.apply(format_clean, axis=1)

# FINAL DATA

In [14]:
safety_df["Unsafety"] = 0
unsafety_df["Unsafety"] = 1
final_data = pd.concat([safety_df, unsafety_df], axis=0, ignore_index=True)
final_data

,input,output,Unsafety,categorical
0,Hãy cho tôi ba mẹo để giữ sức khỏe.,1 Ăn một chế độ ăn uống cân bằng và bổ dưỡng H...,0,NaN
1,Ba màu chính là gì?,"Ba màu chính là đỏ, xanh dương và vàng Những m...",0,NaN
2,Mô tả cấu trúc của một nguyên tử,Một nguyên tử là khối cơ bản của tất cả vật ch...,0,NaN
3,Làm thế nào để giảm ô nhiễm không khí?,Có một số cách để giảm ô nhiễm không khí bao g...,0,NaN
4,Giả sử bạn là một người quản lý dự án của một ...,Tôi phải đưa ra quyết định khó khăn khi tôi là...,0,NaN
...,...,...,...,...
381917,". \n i totally agree, this stuff is nothing bu...",NaN,1,"['toxic', 'severe_toxic', 'obscene', 'threat',..."
381918,== Throw from out field to home plate. == \n\n...,NaN,1,"['toxic', 'severe_toxic', 'obscene', 'threat',..."
381919,""" \n\n == Okinotorishima categories == \n\n I ...",NaN,1,"['toxic', 'severe_toxic', 'obscene', 'threat',..."
381920,""" \n\n == """"One of the founding nations of the...",NaN,1,"['toxic', 'severe_toxic', 'obscene', 'threat',..."


In [68]:
final_data.to_csv("ChatbotVi-Data.csv", index=False)